In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os.path as osp
import re
import ast
from joblib import Parallel, delayed
from cleaning import get_users_for_episode_count_ban, get_usernames_to_keep_cached, get_anime_ids_after_cleaning_cached, user_na_values

In [2]:
# use this to keep the cells wider, very nice on wide screens, set the percentage as you like it
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [3]:
%load_ext autoreload

%autoreload 2

In [4]:
animelists = pd.read_csv('animelists_filtered.csv', na_values=user_na_values, keep_default_na=False)

In [5]:
plt.rcParams.update(mpl.rcParamsDefault)  # for recovering from xkcd

# taken from https://github.com/ctokheim/matplotlibrc
#plt.style.use('seaborn')
plt.style.use('seaborn-talk')
#plt.style.use('ggplot')
#plt.xkcd()

conf = {
    'font.size': 14.0,
    'axes.grid': True,
    'axes.axisbelow': True,
    'axes.edgecolor': 'black',
#    'axes.facecolor': '#E5E5EF',
#    'axes.facecolor': '#E0E0EF',
    'axes.labelcolor': 'black',
    'axes.titlesize': 15.6,
    'axes.labelsize': 'large',    
    'figure.figsize': (12, 7),
    'figure.titlesize': 'x-large',
    'grid.linewidth': 1.3,
    'xtick.labelsize': 'large',
    'ytick.labelsize': 'large',
    'xtick.color': 'black',
    'ytick.color': 'black',
    'legend.fontsize': 'large',
}

plt.rcParams.update(conf)


In [6]:
plt.rcParams

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


RcParams({'_internal.classic_mode': False,
          'agg.path.chunksize': 0,
          'animation.avconv_args': [],
          'animation.avconv_path': 'avconv',
          'animation.bitrate': -1,
          'animation.codec': 'h264',
          'animation.convert_args': [],
          'animation.convert_path': 'convert',
          'animation.embed_limit': 20.0,
          'animation.ffmpeg_args': [],
          'animation.ffmpeg_path': 'ffmpeg',
          'animation.frame_format': 'png',
          'animation.html': 'none',
          'animation.html_args': [],
          'animation.writer': 'ffmpeg',
          'axes.autolimit_mode': 'data',
          'axes.axisbelow': True,
          'axes.edgecolor': 'black',
          'axes.facecolor': 'white',
          'axes.formatter.limits': [-7, 7],
          'axes.formatter.min_exponent': 0,
          'axes.formatter.offset_threshold': 4,
          'axes.formatter.use_locale': False,
          'axes.formatter.use_mathtext': False,
          'axes.for

In [7]:
plt.style.available

['bmh',
 'classic',
 'dark_background',
 'fast',
 'fivethirtyeight',
 'ggplot',
 'grayscale',
 'seaborn-bright',
 'seaborn-colorblind',
 'seaborn-dark-palette',
 'seaborn-dark',
 'seaborn-darkgrid',
 'seaborn-deep',
 'seaborn-muted',
 'seaborn-notebook',
 'seaborn-paper',
 'seaborn-pastel',
 'seaborn-poster',
 'seaborn-talk',
 'seaborn-ticks',
 'seaborn-white',
 'seaborn-whitegrid',
 'seaborn',
 'Solarize_Light2',
 'tableau-colorblind10',
 '_classic_test']

In [8]:
animelists['my_last_updated'] = pd.to_datetime(animelists['my_last_updated'], unit='s')

In [9]:
# some user with weird last online date
animelists[animelists['username'] == 'RedvelvetDaisuki'].head(15)

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
1,RedvelvetDaisuki,21,0,0000-00-00,0000-00-00,0,3,0.0,0,2012-12-14 10:25:01,NaN
95629,RedvelvetDaisuki,120,1,2013-02-02,0000-00-00,3,3,NaN,0,2013-02-02 12:27:53,NaN
212785,RedvelvetDaisuki,269,282,2012-11-00,0000-00-00,6,3,NaN,0,2012-12-30 10:18:22,NaN
276646,RedvelvetDaisuki,721,38,2014-07-27,2014-00-00,8,2,0.0,0,2018-02-09 15:02:20,NaN
288406,RedvelvetDaisuki,853,26,2012-12-24,2012-12-24,9,2,NaN,0,2012-12-24 13:33:51,NaN
387077,RedvelvetDaisuki,1571,0,0000-00-00,0000-00-00,0,6,0.0,0,2013-03-19 03:21:02,NaN
419888,RedvelvetDaisuki,1698,23,2013-03-21,2013-03-22,8,2,NaN,0,2013-03-22 15:38:38,NaN
446319,RedvelvetDaisuki,1735,312,2012-00-00,0000-00-00,9,3,NaN,0,2013-06-05 03:49:42,NaN
566786,RedvelvetDaisuki,3092,12,2012-12-29,2012-12-30,10,2,NaN,0,2013-04-07 15:33:35,"Slice of Life, Comedy, Romance, Yaoi"
604812,RedvelvetDaisuki,4224,25,2014-08-10,2014-08-13,7,2,NaN,0,2014-08-13 03:30:48,NaN


In [10]:
animelists.dtypes

username                       object
anime_id                        int64
my_watched_episodes             int64
my_start_date                  object
my_finish_date                 object
my_score                        int64
my_status                       int64
my_rewatching                 float64
my_rewatching_ep                int64
my_last_updated        datetime64[ns]
my_tags                        object
dtype: object

In [11]:
grouped_lists = animelists.groupby('username')

In [12]:
%%timeit
# some users have weird last online date, but have correct last update in their animelist, so I can infer it from that.
#Now I am just trying it
#animelists[animelists['username'] == 'RedvelvetDaisuki']['my_last_updated'].max()
# 3.2 s ± 38.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
grouped_lists.get_group('RedvelvetDaisuki')['my_last_updated'].max()
# 991 µs ± 72.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
# so once grouping and then queying group per username is much faster than performing search each time

756 µs ± 23.7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
animelists['my_watched_episodes'].hist(bins=500, figsize=(16, 8))
plt.gca().set_yscale('log')


In [14]:
print('unique anime before:', animelists['anime_id'].unique().shape)
print('animelists before:', animelists.shape)
# some users have insanely large amount of episodes, list and ban them
users_for_ban = get_users_for_episode_count_ban(animelists)
# banning users with 65535 episodes of single anime in their list -> not trustworthy
print(users_for_ban)
print('num of users for ban:', len(users_for_ban))
print('num of records with 65535 episodes filled:', (animelists['my_watched_episodes'] == 65535).sum())



unique anime before: (14474,)
animelists before: (35802006, 11)
0        Gothicbabe26
1         LeoAnimeMan
2         narutofan69
3             Angel_D
4      dheiin_society
5             heizwei
6             warrose
7     Takehara_Minami
8             ozieboy
9          YoruNoMajo
10               Xiet
11         Kihara-Kun
12            donakos
13          Amaru-kun
14       Archeologist
15    konan_akatsuki_
16             MoiZzZ
17     TheDerpyMuffin
18              ajkla
19        zaturama007
20           thomasrm
21         The-Marker
22             Taizai
23         Koska-chan
24      asswithnolife
dtype: object
num of users for ban: 25
num of records with 65535 episodes filled: 1863


In [15]:
animelists = animelists[~animelists['username'].isin(users_for_ban)]
animelists = animelists.reset_index(drop=True)  # so indices are aligned, otherwise it breaks when using same mask with other dataframe
print('unique anime after:', animelists['anime_id'].unique().shape)
print('animelists after:', animelists.shape)

unique anime after: (14474,)
animelists after: (35793977, 11)


In [16]:
animelists['my_watched_episodes'].hist(bins=5000, figsize=(16, 8))
plt.gca().set_yscale('log')


In [17]:
animes = pd.read_csv('anime_filtered.csv')
animelists_with_eps = pd.merge(animelists, animes[['anime_id', 'episodes', 'title']], on='anime_id')

In [18]:
display(animelists_with_eps.shape)
display(animelists.shape)

(35793977, 13)

(35793977, 11)

In [19]:
animelists_with_eps.head()

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags,episodes,title
0,karthiga,21,586,0000-00-00,0000-00-00,9,1,NaN,0,2013-03-03 10:52:53,NaN,0,One Piece
1,RedvelvetDaisuki,21,0,0000-00-00,0000-00-00,0,3,0.0,0,2012-12-14 10:25:01,NaN,0,One Piece
2,Damonashu,21,418,0000-00-00,0000-00-00,10,1,0.0,0,2009-09-30 07:39:05,NaN,0,One Piece
3,bskai,21,75,0000-00-00,0000-00-00,8,1,0.0,0,2010-06-15 21:31:23,NaN,0,One Piece
4,Slimak,21,834,0000-00-00,0000-00-00,10,1,0.0,0,2018-05-01 12:05:21,NaN,0,One Piece


In [20]:
# just showing shitheads with incorrectly filled episodes
err_mask = (animelists_with_eps['my_watched_episodes'] > animelists_with_eps['episodes']) & (animelists_with_eps['episodes'] != 0)
print('num of incorrect records: ', err_mask.sum())
print('num of users with incorrect records: ', len(animelists_with_eps[err_mask]['username'].unique()))
animelists_with_eps[err_mask].head()

num of incorrect records:  97931
num of users with incorrect records:  45905


,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags,episodes,title
50353,heeroYuy,59,43,0000-00-00,0000-00-00,9,6,0.0,0,1970-01-01 00:00:00,NaN,26,Chobits
62647,Sieg83,59,27,0000-00-00,0000-00-00,10,2,0.0,0,2008-04-29 01:41:44,NaN,26,Chobits
65431,ultimaweaponx,59,27,0000-00-00,0000-00-00,8,2,0.0,0,1970-01-01 00:00:00,NaN,26,Chobits
67599,Yashima,59,9001,0000-00-00,0000-00-00,7,2,NaN,0,2014-09-02 14:32:43,NaN,26,Chobits
74194,naturefull,59,28,0000-00-00,0000-00-00,8,2,0.0,0,2009-08-12 21:05:02,NaN,26,Chobits


In [21]:
display(animelists_with_eps.dtypes)
display(animelists.dtypes)
display(animelists_with_eps.shape)
display(animelists.shape)
display(animelists_with_eps.index)
display(animelists.index)
display(animelists_with_eps[err_mask].shape)
display(animelists[err_mask].shape)

username                       object
anime_id                        int64
my_watched_episodes             int64
my_start_date                  object
my_finish_date                 object
my_score                        int64
my_status                       int64
my_rewatching                 float64
my_rewatching_ep                int64
my_last_updated        datetime64[ns]
my_tags                        object
episodes                        int64
title                          object
dtype: object

username                       object
anime_id                        int64
my_watched_episodes             int64
my_start_date                  object
my_finish_date                 object
my_score                        int64
my_status                       int64
my_rewatching                 float64
my_rewatching_ep                int64
my_last_updated        datetime64[ns]
my_tags                        object
dtype: object

(35793977, 13)

(35793977, 11)

Int64Index([       0,        1,        2,        3,        4,        5,
                   6,        7,        8,        9,
            ...
            35793967, 35793968, 35793969, 35793970, 35793971, 35793972,
            35793973, 35793974, 35793975, 35793976],
           dtype='int64', length=35793977)

RangeIndex(start=0, stop=35793977, step=1)

(97931, 13)

(97931, 11)

In [22]:
display(err_mask.size)
animelists.size

35793977

393733747

In [23]:
# fixing incorrectly filled my_watched_episodes
filtered_eps = animelists_with_eps[err_mask]['episodes']
animelists.loc[err_mask, 'my_watched_episodes'] = filtered_eps

In [24]:
animelists[err_mask]

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
50353,heeroYuy,59,26,0000-00-00,0000-00-00,9,6,0.0,0,1970-01-01 00:00:00,NaN
62647,Sieg83,59,26,0000-00-00,0000-00-00,10,2,0.0,0,2008-04-29 01:41:44,NaN
65431,ultimaweaponx,59,26,0000-00-00,0000-00-00,8,2,0.0,0,1970-01-01 00:00:00,NaN
67599,Yashima,59,26,0000-00-00,0000-00-00,7,2,NaN,0,2014-09-02 14:32:43,NaN
74194,naturefull,59,26,0000-00-00,0000-00-00,8,2,0.0,0,2009-08-12 21:05:02,NaN
75238,Sakyosa,59,26,0000-00-00,0000-00-00,0,2,0.0,0,2008-12-01 21:18:25,NaN
76289,NC7E,59,26,0000-00-00,0000-00-00,6,4,0.0,0,2013-01-21 22:28:27,NaN
84822,Niero17,59,26,0000-00-00,0000-00-00,9,2,0.0,0,2008-04-25 10:59:30,NaN
100680,MurTi,120,26,2014-00-00,2014-00-00,6,2,0.0,0,2014-08-31 17:07:54,NaN
112321,Yashima,120,26,0000-00-00,0000-00-00,10,2,NaN,0,2014-09-02 14:34:44,NaN


In [25]:
# example of watch time of some user with fixed episodes cound and thus watch time needs fix too.
# So watch time will be tested and reconstructed in this part
#users[users['username'] == 'heeroYuy']

In [26]:
#animelists[animelists['username'] == 'heeroYuy']

In [27]:
animes.columns

Index(['anime_id', 'title', 'title_english', 'title_japanese',
       'title_synonyms', 'image_url', 'type', 'source', 'episodes', 'status',
       'airing', 'aired_string', 'aired', 'duration', 'rating', 'score',
       'scored_by', 'rank', 'popularity', 'members', 'favorites', 'background',
       'premiered', 'broadcast', 'related', 'producer', 'licensor', 'studio',
       'genre', 'opening_theme', 'ending_theme'],
      dtype='object')

In [28]:
animes[['anime_id', 'title', 'episodes', 'duration']].head()

,anime_id,title,episodes,duration
0,11013,Inu x Boku SS,12,24 min. per ep.
1,2104,Seto no Hanayome,26,24 min. per ep.
2,5262,Shugo Chara!! Doki,51,24 min. per ep.
3,721,Princess Tutu,38,16 min. per ep.
4,12365,Bakuman. 3rd Season,25,24 min. per ep.


In [29]:
#animes['duration'].value_counts()

In [30]:
# calculating watch time in minutes and adding it to new column for anime
def str_to_minutes(string):
    m = re.search('((\d+) hr\.)?( ?(\d+) min\.)?( ?(\d+) sec\.)?( per ep\.)?', string)
    hours = 0 if m.group(2) is None else int(m.group(2))
    minutes = 0 if m.group(4) is None else int(m.group(4))
    seconds = 0 if m.group(6) is None else int(m.group(6))
    return hours * 60 + minutes + seconds / 60

animes['duration_min'] = animes['duration'].apply(str_to_minutes)

In [31]:
animes[['anime_id', 'title', 'episodes', 'duration', 'duration_min']].head()

,anime_id,title,episodes,duration,duration_min
0,11013,Inu x Boku SS,12,24 min. per ep.,24.0
1,2104,Seto no Hanayome,26,24 min. per ep.,24.0
2,5262,Shugo Chara!! Doki,51,24 min. per ep.,24.0
3,721,Princess Tutu,38,16 min. per ep.,16.0
4,12365,Bakuman. 3rd Season,25,24 min. per ep.,24.0


In [32]:
lists_with_mins = pd.merge(animelists, animes[['anime_id', 'duration_min', 'title']], on='anime_id')

In [33]:
one_user_data = lists_with_mins[lists_with_mins['username'] == 'heeroYuy']
one_user_data.head()

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags,duration_min,title
50353,heeroYuy,59,26,0000-00-00,0000-00-00,9,6,0.0,0,1970-01-01 00:00:00,NaN,24.0,Chobits
87470,heeroYuy,74,26,0000-00-00,0000-00-00,10,2,0.0,0,2007-11-07 13:32:49,NaN,25.0,Gakuen Alice
128310,heeroYuy,178,19,0000-00-00,0000-00-00,8,1,0.0,0,2007-11-07 14:13:10,NaN,24.0,Ultra Maniac
133661,heeroYuy,210,161,0000-00-00,0000-00-00,10,2,0.0,0,2007-11-07 13:47:44,NaN,24.0,Ranma ½
150736,heeroYuy,232,40,0000-00-00,0000-00-00,10,6,0.0,0,1970-01-01 00:00:00,NaN,25.0,Cardcaptor Sakura


In [34]:
(one_user_data['my_watched_episodes'] * one_user_data['duration_min']).sum() / (60 * 24) # because it is in days

38.00069444444444

In [35]:
one_user_data = lists_with_mins[lists_with_mins['username'] == 'abystoma2']
(one_user_data['my_watched_episodes'] * one_user_data['duration_min']).sum() / (60 * 24) # because it is in days

623.2052662037037

In [36]:
one_user_data = lists_with_mins[lists_with_mins['username'] == 'warrose']
(one_user_data['my_watched_episodes'] * one_user_data['duration_min']).sum() / (60 * 24) # because it is in days

0.0

In [37]:
# some weitd shit happening with grouped_tuples, they are insanely slow
grouped_lists = lists_with_mins.groupby('username')
#grouped_tuples = [(name, group) for name, group in grouped_lists]
#grouped_dict = {name: group for name, group in grouped_lists}

In [38]:
#[(i, j) for i, j in grouped_lists.groups]
#grouped_lists.groups[list(grouped_lists.groups.keys())[0]]
lists_with_mins.loc[grouped_lists.groups[list(grouped_lists.groups.keys())[0]]]

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags,duration_min,title
16725,----phoebelyn,21,0,0000-00-00,0000-00-00,0,1,0.0,0,1970-01-01 00:00:00,NaN,24.0,One Piece
108291,----phoebelyn,120,0,0000-00-00,0000-00-00,0,6,0.0,0,1970-01-01 00:00:00,NaN,24.0,Fruits Basket
306755,----phoebelyn,853,26,0000-00-00,0000-00-00,0,2,0.0,0,2008-02-18 22:11:38,NaN,23.0,Ouran Koukou Host Club
369540,----phoebelyn,957,39,0000-00-00,0000-00-00,0,2,0.0,0,1970-01-01 00:00:00,NaN,25.0,Saiunkoku Monogatari
395563,----phoebelyn,1571,0,0000-00-00,0000-00-00,0,3,0.0,0,1970-01-01 00:00:00,NaN,25.0,Ghost Hunt
413247,----phoebelyn,1579,0,0000-00-00,0000-00-00,10,3,0.0,0,1970-01-01 00:00:00,NaN,24.0,Kiniro no Corda: Primo Passo
430176,----phoebelyn,1698,0,0000-00-00,0000-00-00,0,1,0.0,0,1970-01-01 00:00:00,NaN,22.0,Nodame Cantabile
465298,----phoebelyn,1735,0,0000-00-00,0000-00-00,0,1,0.0,0,1970-01-01 00:00:00,NaN,23.0,Naruto: Shippuuden
1204819,----phoebelyn,1,0,0000-00-00,0000-00-00,0,1,0.0,0,1970-01-01 00:00:00,NaN,24.0,Cowboy Bebop
1286404,----phoebelyn,19,0,0000-00-00,0000-00-00,0,1,0.0,0,1970-01-01 00:00:00,NaN,24.0,Monster


In [39]:
def count_to_days(group):
    return (group['my_watched_episodes'] * group['duration_min']).sum() / (60 * 24)

# just can not get it fucking faster
def apply_parallel(df, func):
    ret_st = Parallel(n_jobs=4, backend='threading')(delayed(func)(row) for row in df)
    #return pd.concat(ret_st)
    return ret_st

def apply_parallel_2(df_grouped, func):
    ret_st = Parallel(n_jobs=4, backend='threading')(delayed(func)(group) for name, group in df_grouped)
    #return pd.Series(np.concatenate(ret_st))
    return ret_st

def apply_parallel_3(df_grouped, func):
    ret_st = Parallel(n_jobs=4, backend='threading')(delayed(func)(group.copy()) for name, group in df_grouped)
    #return pd.Series(np.concatenate(ret_st))
    return ret_st

def apply_parallel_4(df_grouped, func):
    ret_st = Parallel(n_jobs=4, backend='threading')(delayed(func)(group) for group in df_grouped)
    #return pd.Series(np.concatenate(ret_st))
    return ret_st


In [40]:
usernames_arr = animelists['username'].unique()
usernames = pd.Series(data=usernames_arr, index=usernames_arr)

In [41]:
'''
%%time
user_days_spent_watching = usernames[:10000].apply(lambda x: count_to_days(lists_with_mins.loc[grouped_lists.groups[x]]))
%%time
user_days_spent_watching = usernames[:10000].apply(lambda x: count_to_days(grouped_lists.get_group(x)))
%%time
user_days_spent_watching = usernames[:1000].apply(lambda x: count_to_days(grouped_dict[x]))
%%time
user_days_spent_watching_2 = apply_parallel(usernames[:10000], lambda x: count_to_days(grouped_lists.get_group(x)))
%%time
user_days_spent_watching_2 = apply_parallel(usernames[:10000], lambda x: count_to_days(lists_with_mins.loc[grouped_lists.groups[x]]))
%%time
user_days_spent_watching_2 = apply_parallel(usernames[:10000], lambda x: count_to_days(lists_with_mins.loc[grouped_lists.groups[x]].copy()))
%%time
#user_days_spent_watching_2 = apply_parallel_2(grouped_lists, lambda group: count_to_days(group))
user_days_spent_watching_2 = apply_parallel_2(grouped_tuples[:1000], count_to_days)
%%time
user_days_spent_watching_3 = apply_parallel_3(grouped_lists, lambda group: count_to_days(group))
%%time
user_days_spent_watching_4 = apply_parallel_3(grouped_dict.values(), lambda group: count_to_days(group))
%%time
#[(name, type(group)) for name, group in grouped_lists]
#[(name, type(group)) for name, group in grouped_tuples]
[type(group) for group in grouped_dict.values()]
#for name, group in grouped_lists:
#    pass
#for username in usernames:
#    pass
'''

'\n%%time\nuser_days_spent_watching = usernames[:10000].apply(lambda x: count_to_days(lists_with_mins.loc[grouped_lists.groups[x]]))\n%%time\nuser_days_spent_watching = usernames[:10000].apply(lambda x: count_to_days(grouped_lists.get_group(x)))\n%%time\nuser_days_spent_watching = usernames[:1000].apply(lambda x: count_to_days(grouped_dict[x]))\n%%time\nuser_days_spent_watching_2 = apply_parallel(usernames[:10000], lambda x: count_to_days(grouped_lists.get_group(x)))\n%%time\nuser_days_spent_watching_2 = apply_parallel(usernames[:10000], lambda x: count_to_days(lists_with_mins.loc[grouped_lists.groups[x]]))\n%%time\nuser_days_spent_watching_2 = apply_parallel(usernames[:10000], lambda x: count_to_days(lists_with_mins.loc[grouped_lists.groups[x]].copy()))\n%%time\n#user_days_spent_watching_2 = apply_parallel_2(grouped_lists, lambda group: count_to_days(group))\nuser_days_spent_watching_2 = apply_parallel_2(grouped_tuples[:1000], count_to_days)\n%%time\nuser_days_spent_watching_3 = apply

In [42]:
if not osp.isfile('cache_user_days_spent_watching.csv'):
    user_days_spent_watching = usernames.apply(lambda x: count_to_days(lists_with_mins.loc[grouped_lists.groups[x]]))
    user_days_spent_watching.to_frame(name='days_spent_watching').to_csv('cache_user_days_spent_watching.csv', sep=',', encoding='utf-8')

if not osp.isfile('cache_users_last_list_update.csv'):
    users_last_list_update = usernames.apply(lambda x: grouped_lists.get_group(x)['my_last_updated'].max())
    users_last_list_update.to_frame(name='last_list_update').to_csv('cache_users_last_list_update.csv', sep=',', encoding='utf-8')
    
if not osp.isfile('cache_users_stats_episodes.csv'):
    users_stats_episodes = usernames.apply(lambda x: grouped_lists.get_group(x)['my_watched_episodes'].sum())
    users_stats_episodes.to_frame(name='stats_episodes').to_csv('cache_users_stats_episodes.csv', sep=',', encoding='utf-8')

In [43]:
(lists_with_mins['username'] == 'null').sum()

261

In [44]:
animelists['username'].isna().sum()

0

In [45]:
#animelists_with_eps[animelists['username'] == 'dheiin_society']['my_watched_episodes'].nlargest(10)
animelists_with_eps[animelists['username'] == 'ComfyLoli'].sort_values(by='my_watched_episodes').head()

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags,episodes,title
35788089,ComfyLoli,37481,0,0000-00-00,0000-00-00,0,6,0.0,0,2018-05-05 12:17:40,NaN,0,Ongaku Shoujo (TV)
34901467,ComfyLoli,28613,0,0000-00-00,0000-00-00,0,6,0.0,0,2014-12-17 16:44:26,NaN,0,Ginga Jinpuu Jinraiger
34900614,ComfyLoli,27785,0,0000-00-00,0000-00-00,0,6,NaN,0,2014-10-12 14:18:03,NaN,13,Doamaiger D
34899707,ComfyLoli,22465,0,0000-00-00,0000-00-00,0,6,0.0,0,2014-11-30 16:01:23,NaN,48,KutsuDaru.
34898326,ComfyLoli,21851,0,0000-00-00,0000-00-00,0,6,NaN,0,2015-04-29 13:56:37,NaN,12,Tesagure! Bukatsumono Encore


In [46]:
animelists.describe()

,anime_id,my_watched_episodes,my_score,my_status,my_rewatching,my_rewatching_ep
count,3.579398e+07,3.579398e+07,3.579398e+07,3.579398e+07,2.802716e+07,3.579398e+07
mean,1.169565e+04,1.213340e+01,4.538777e+00,3.038835e+00,7.663282e-04,1.605091e-01
std,1.137970e+04,3.612339e+01,3.915338e+00,1.746322e+00,2.767202e-02,9.434558e+02
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.690000e+03,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
50%,7.870000e+03,3.000000e+00,6.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
75%,1.967100e+04,1.300000e+01,8.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00
max,3.791600e+04,9.999000e+03,1.000000e+01,5.500000e+01,1.000000e+00,5.644513e+06


In [47]:
animelists.head()

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
0,karthiga,21,586,0000-00-00,0000-00-00,9,1,NaN,0,2013-03-03 10:52:53,NaN
1,RedvelvetDaisuki,21,0,0000-00-00,0000-00-00,0,3,0.0,0,2012-12-14 10:25:01,NaN
2,Damonashu,21,418,0000-00-00,0000-00-00,10,1,0.0,0,2009-09-30 07:39:05,NaN
3,bskai,21,75,0000-00-00,0000-00-00,8,1,0.0,0,2010-06-15 21:31:23,NaN
4,Slimak,21,834,0000-00-00,0000-00-00,10,1,0.0,0,2018-05-01 12:05:21,NaN


In [48]:
usernames_to_keep = get_usernames_to_keep_cached()
# clearing animelists according to dropped users
animelists_smaller = pd.merge(animelists, usernames_to_keep, on='username', how='right')
animelists_smaller.describe()

,anime_id,my_watched_episodes,my_score,my_status,my_rewatching,my_rewatching_ep
count,3.370156e+07,3.370156e+07,3.370156e+07,3.370156e+07,2.641755e+07,3.370156e+07
mean,1.170690e+04,1.219264e+01,4.555682e+00,3.040442e+00,7.560882e-04,1.701883e-01
std,1.138307e+04,3.627464e+01,3.923886e+00,1.748151e+00,2.748666e-02,9.723027e+02
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.690000e+03,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
50%,8.023000e+03,3.000000e+00,6.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
75%,1.968500e+04,1.300000e+01,8.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00
max,3.791600e+04,9.999000e+03,1.000000e+01,5.500000e+01,1.000000e+00,5.644513e+06


In [49]:
animelists.describe()

,anime_id,my_watched_episodes,my_score,my_status,my_rewatching,my_rewatching_ep
count,3.579398e+07,3.579398e+07,3.579398e+07,3.579398e+07,2.802716e+07,3.579398e+07
mean,1.169565e+04,1.213340e+01,4.538777e+00,3.038835e+00,7.663282e-04,1.605091e-01
std,1.137970e+04,3.612339e+01,3.915338e+00,1.746322e+00,2.767202e-02,9.434558e+02
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.690000e+03,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
50%,7.870000e+03,3.000000e+00,6.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
75%,1.967100e+04,1.300000e+01,8.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00
max,3.791600e+04,9.999000e+03,1.000000e+01,5.500000e+01,1.000000e+00,5.644513e+06


In [50]:
animelists.head()

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
0,karthiga,21,586,0000-00-00,0000-00-00,9,1,NaN,0,2013-03-03 10:52:53,NaN
1,RedvelvetDaisuki,21,0,0000-00-00,0000-00-00,0,3,0.0,0,2012-12-14 10:25:01,NaN
2,Damonashu,21,418,0000-00-00,0000-00-00,10,1,0.0,0,2009-09-30 07:39:05,NaN
3,bskai,21,75,0000-00-00,0000-00-00,8,1,0.0,0,2010-06-15 21:31:23,NaN
4,Slimak,21,834,0000-00-00,0000-00-00,10,1,0.0,0,2018-05-01 12:05:21,NaN


In [51]:
print('unique anime before filtering: ', animelists['anime_id'].unique().shape)
print('unique anime after filtering: ', animelists_smaller['anime_id'].unique().shape)

unique anime before filtering:  (14474,)
unique anime after filtering:  (14402,)


In [52]:
# now replacing data with cleaned animelists, using only columns before merge
animelists = animelists_smaller[animelists.columns]

In [53]:
pd.Series(animelists_smaller['anime_id'].unique()).to_frame(name='anime_id').to_csv('cache_anime_ids_after_username_clear.csv', sep=',', index=False, encoding='utf-8')

In [54]:
anime_ids_to_remain = get_anime_ids_after_cleaning_cached()

In [55]:
animelists[~animelists['anime_id'].isin(anime_ids_to_remain['anime_id'])].shape

(2417776, 11)

In [56]:
animelists = animelists[animelists['anime_id'].isin(anime_ids_to_remain['anime_id'])]

In [57]:
animelists.shape

(31283787, 11)

In [ ]:
animelists.describe()

In [59]:
animelists['username'].unique().shape
# no users rated only these truncated anime, so no need to clear users now

(108709,)

In [60]:
animelists.to_csv('animelists_cleaned_2.csv', index=False, sep=',', encoding='utf-8')